# PyCity Schools Analysis
##Homework Week 4


In [31]:
# Dependencies and Setup
import pandas as pd
import numpy as np


In [32]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_data_to_load)
student_df = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_student_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])


### District Summary

* Create a high level snapshot (in table form) of the district's key metrics, including:
  * Total Schools
  * Total Students
  * Total Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [33]:
# School - DataFrame Direct Calc
SchoolCount = school_df["School ID"].count()
TotalBudget = school_df["budget"].sum()

# Student - DataFrame Direct Calc
StudentCount = student_df["Student ID"].count()
AverageMath = student_df["math_score"].mean()
AverageReading = student_df["reading_score"].mean()

In [34]:
# Derived Calculation
AverageOverall = (AverageMath + AverageReading)/2

# Calculate the percentage of students with a passing math score (70 or greater)

MathGT70Count = student_df["Student ID"].loc[student_df["math_score"] >= 70].count()
MathGT70Percent = (MathGT70Count /StudentCount)*100

ReadingGT70Percent = (student_df["Student ID"].loc[student_df["reading_score"] >= 70].count()/StudentCount)*100


In [35]:
# Creating a summary DataFrame using above values
district_summary_df = pd.DataFrame({"Total Schools":  [SchoolCount],
                           "Total Students":   format(StudentCount, ",.0f"),
                           "Total Budget": "$" + (format(TotalBudget, ",.2f")),
                           "Average Math Score": [AverageMath],
                           "Average Reading Score": [AverageReading],
                           "% Passing Math":  [MathGT70Percent],
                           "% Passing Reading":  [ReadingGT70Percent],
                           "% Overall Passing Rate": [AverageOverall]
                          })

district_summary_df


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


### School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [36]:

std_sub_col_df = student_df[["school_name", "student_name", "math_score", "reading_score", "grade"]]

# . add boolean column for math and reading pass
std_sub_col_df = std_sub_col_df.assign(math_pass = True, reading_pass = True)

# . update math and reading pass accordingly
std_sub_col_df["math_pass"] = np.where(std_sub_col_df["math_score"] < 70, False, True)
std_sub_col_df["reading_pass"] = np.where(std_sub_col_df["reading_score"] < 70, False, True)


In [37]:
# Math nd Reading averages for Schools
std_math_avg_gdf = std_sub_col_df.groupby(["school_name"])["math_score"].mean().reset_index().rename(columns={'math_score': 'math_average'})
std_reading_avg_gdf =  std_sub_col_df.groupby(["school_name"])["reading_score"].mean().reset_index().rename(columns={'reading_score': 'reading_average'})

# Math nd Reading pass count for Schools
std_math_pass_gdf = std_sub_col_df[std_sub_col_df.math_pass].groupby("school_name")["math_score"].count().reset_index().rename(columns={'math_score': 'math_pass_count'})
std_reading_pass_gdf = std_sub_col_df[std_sub_col_df.reading_pass].groupby("school_name")["reading_score"].count().reset_index().rename(columns={'reading_score': 'reading_pass_count'})


std_pass_gdf = std_math_pass_gdf.merge(std_reading_pass_gdf, on = "school_name")

std_calc_df = std_pass_gdf.merge(std_math_avg_gdf, on = "school_name")
std_calc_df = std_calc_df.merge(std_reading_avg_gdf, on = "school_name")
std_calc_df = std_calc_df.merge(school_df, on = "school_name")
std_calc_df = std_calc_df.assign(math_pass_percent = (std_calc_df["math_pass_count"]/std_calc_df["size"])*100,
                                 reading_pass_percent = (std_calc_df["reading_pass_count"]/std_calc_df["size"])*100)
std_calc_df = std_calc_df.assign(overall_passing_rate = (std_calc_df["math_pass_percent"]+ std_calc_df["reading_pass_percent"])/2,
                                 per_student_budget = std_calc_df["budget"]/std_calc_df["size"])
# school_student_calc_df = std_calc_df.merge(std_sub_col_df[["school_name", "grade"]], on = "school_name")
school_student_calc_df = std_calc_df


In [38]:
# Creating a summary DataFrame using values calculated in school_student_calc_df
school_summary_df = pd.DataFrame({"School Name":  school_student_calc_df["school_name"],
                           "School Type":   school_student_calc_df["type"],
                           "Total Students": school_student_calc_df["size"],
                           "Total School Budget": school_student_calc_df["budget"],
                           "Per Student Budget": school_student_calc_df["per_student_budget"],
                           "Average Math Score": school_student_calc_df["math_average"],
                           "Average Reading Score": school_student_calc_df["reading_average"],
                           "% Passing Math":  school_student_calc_df["math_pass_percent"],
                           "% Passing Reading":  school_student_calc_df["reading_pass_percent"],
                           "% Overall Passing Rate": school_student_calc_df["overall_passing_rate"]
                          })
school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
5,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
7,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
8,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


### Top Performing Schools (By Passing Rate)

* Create a table that highlights the top 5 performing schools based on Overall Passing Rate. Include:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [43]:
school_top_five_df = std_calc_df.sort_values(by=["overall_passing_rate"], ascending=False).reset_index()
school_top_five_df = school_top_five_df.head(5)

In [89]:
# Creating a summary DataFrame using values calculated in school_top_five_df
top5_school_summary_df = pd.DataFrame({"School Name":  school_top_five_df["school_name"],
                           "School Type":   school_top_five_df["type"],
                           "Total Students": school_top_five_df["size"],
                           "Total School Budget": list(map('${:.2f}'.format,school_top_five_df["budget"])),
                           "Per Student Budget": list(map('${:.2f}'.format,school_top_five_df["per_student_budget"])),
                           "Average Math Score": list(map('{:.2f}'.format,school_top_five_df["math_average"])),
                           "Average Reading Score": list(map('{:.2f}'.format,school_top_five_df["reading_average"])),
                           "% Passing Math":  list(map('{:.2f}%'.format,school_top_five_df["math_pass_percent"])),
                           "% Passing Reading":  list(map('{:.2f}%'.format,school_top_five_df["reading_pass_percent"])),
                           "% Overall Passing Rate": list(map('{:.4f}%'.format,school_top_five_df["overall_passing_rate"]))
                          })
top5_school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Cabrera High School,Charter,1858,$1081356.00,$582.00,83.06,83.98,94.13%,97.04%,95.5867%
1,Thomas High School,Charter,1635,$1043130.00,$638.00,83.42,83.85,93.27%,97.31%,95.2905%
2,Pena High School,Charter,962,$585858.00,$609.00,83.84,84.04,94.59%,95.95%,95.2703%
3,Griffin High School,Charter,1468,$917500.00,$625.00,83.35,83.82,93.39%,97.14%,95.2657%
4,Wilson High School,Charter,2283,$1319574.00,$578.00,83.27,83.99,93.87%,96.54%,95.2037%


### Bottom Performing Schools (By Passing Rate)

* Create a table that highlights the bottom 5 performing schools based on Overall Passing Rate. Include all of the same metrics as above.

In [45]:
school_bottom_five_df = std_calc_df.sort_values(by=["overall_passing_rate"], ascending=True).reset_index()
school_bottom_five_df = school_bottom_five_df.head(5)

In [90]:
# Creating a summary DataFrame using values calculated in school_bottom_five_df
bottom5_school_summary_df = pd.DataFrame({"School Name":  school_bottom_five_df["school_name"],
                           "School Type":   school_bottom_five_df["type"],
                           "Total Students": school_bottom_five_df["size"],
                           "Total School Budget": list(map('${:.2f}'.format,school_bottom_five_df["budget"])),
                           "Per Student Budget": list(map('${:.2f}'.format,school_bottom_five_df["per_student_budget"])),
                           "Average Math Score": list(map('{:.2f}'.format,school_bottom_five_df["math_average"])),
                           "Average Reading Score": list(map('{:.2f}'.format,school_bottom_five_df["reading_average"])),
                           "% Passing Math":  list(map('{:.2f}%'.format,school_bottom_five_df["math_pass_percent"])),
                           "% Passing Reading":  list(map('{:.2f}%'.format,school_bottom_five_df["reading_pass_percent"])),
                           "% Overall Passing Rate": list(map('{:.2f}%'.format,school_bottom_five_df["overall_passing_rate"]))
                          })
bottom5_school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Rodriguez High School,District,3999,$2547363.00,$637.00,76.84,80.74,66.37%,80.22%,73.29%
1,Figueroa High School,District,2949,$1884411.00,$639.00,76.71,81.16,65.99%,80.74%,73.36%
2,Huang High School,District,2917,$1910635.00,$655.00,76.63,81.18,65.68%,81.32%,73.50%
3,Johnson High School,District,4761,$3094650.00,$650.00,77.07,80.97,66.06%,81.22%,73.64%
4,Ford High School,District,2739,$1763916.00,$644.00,77.10,80.75,68.31%,79.30%,73.80%


### Math Scores by Grade\*\*

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [53]:
math_score_by_grade = std_sub_col_df.groupby(["school_name", "student_name"])["math_score"].mean().reset_index().rename(columns={'math_score': 'student_math_average'})
math_score_by_grade = math_score_by_grade.merge(std_sub_col_df[["school_name", "student_name"]], on = ["school_name", "student_name"])


In [55]:
# Creating a summary DataFrame using values calculated in math_score_by_grade
math_score_by_grade_df = pd.DataFrame({"School Name":  math_score_by_grade["school_name"],
                                       "Student Name":   math_score_by_grade["student_name"],
                                       "Student Math Average": math_score_by_grade["student_math_average"]
                                      })
math_score_by_grade_df

,School Name,Student Name,Student Math Average
0,Bailey High School,Aaron Atkinson,77.0
1,Bailey High School,Aaron Bailey,70.0
2,Bailey High School,Aaron Carter,85.0
3,Bailey High School,Aaron Clark,55.0
4,Bailey High School,Aaron Cook,67.0
5,Bailey High School,Aaron Cruz,95.0
6,Bailey High School,Aaron Davis Jr.,63.0
7,Bailey High School,Aaron Evans,76.0
8,Bailey High School,Aaron Garcia,98.0
9,Bailey High School,Aaron Gonzalez,60.0


### Reading Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.


In [56]:

reading_score_by_grade = std_sub_col_pd.groupby(["school_name", "student_name"])["reading_score"].mean().reset_index().rename(columns={'reading_score': 'student_reading_average'})
reading_score_by_grade = reading_score_by_grade.merge(std_sub_col_pd[["school_name", "student_name"]], on = ["school_name", "student_name"])


In [57]:
# Creating a summary DataFrame using values calculated in reading_score_by_grade
reading_score_by_grade_df = pd.DataFrame({"School Name":  reading_score_by_grade["school_name"],
                                       "Student Name":   reading_score_by_grade["student_name"],
                                       "Student Reading Average": reading_score_by_grade["student_reading_average"]
                                      })
reading_score_by_grade_df

,School Name,Student Name,Student Reading Average
0,Bailey High School,Aaron Atkinson,88.0
1,Bailey High School,Aaron Bailey,98.0
2,Bailey High School,Aaron Carter,78.0
3,Bailey High School,Aaron Clark,81.0
4,Bailey High School,Aaron Cook,93.0
5,Bailey High School,Aaron Cruz,70.0
6,Bailey High School,Aaron Davis Jr.,97.0
7,Bailey High School,Aaron Evans,93.0
8,Bailey High School,Aaron Garcia,96.0
9,Bailey High School,Aaron Gonzalez,81.0


### Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [61]:
bins = [575, 599, 625, 650, 675]
bin_names = ["<600", "600-625", "626-650", ">650"]

student_spending_bin_df = school_student_calc_df
student_spending_bin_df = student_spending_bin_df.assign(spending_bracket = pd.cut(school_student_calc_df["per_student_budget"], bins, labels=bin_names))


In [91]:
# Creating a summary DataFrame using values calculated in student_spending_bin_df
school_spending_df = pd.DataFrame({"Per Student Spending Bracket": student_spending_bin_df["spending_bracket"],
                                   "School Name":  student_spending_bin_df["school_name"],
                                   "Average Math Score": list(map('{:.2f}'.format,student_spending_bin_df["math_average"])),
                                   "Average Reading Score": list(map('{:.2f}'.format,student_spending_bin_df["reading_average"])),
                                   "% Passing Math":  list(map('{:.2f}%'.format,student_spending_bin_df["math_pass_percent"])),
                                   "% Passing Reading":  list(map('{:.2f}%'.format,student_spending_bin_df["reading_pass_percent"])),
                                   "% Overall Passing Rate": list(map('{:.2f}%'.format,student_spending_bin_df["overall_passing_rate"]))
                                   
                                  })
school_spending_df

,Per Student Spending Bracket,School Name,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,626-650,Bailey High School,77.05,81.03,66.68%,81.93%,74.31%
1,<600,Cabrera High School,83.06,83.98,94.13%,97.04%,95.59%
2,626-650,Figueroa High School,76.71,81.16,65.99%,80.74%,73.36%
3,626-650,Ford High School,77.10,80.75,68.31%,79.30%,73.80%
4,600-625,Griffin High School,83.35,83.82,93.39%,97.14%,95.27%
5,>650,Hernandez High School,77.29,80.93,66.75%,80.86%,73.81%
6,<600,Holden High School,83.80,83.81,92.51%,96.25%,94.38%
7,>650,Huang High School,76.63,81.18,65.68%,81.32%,73.50%
8,626-650,Johnson High School,77.07,80.97,66.06%,81.22%,73.64%
9,600-625,Pena High School,83.84,84.04,94.59%,95.95%,95.27%


### Scores by School Size

* Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).

In [93]:
bins = [0, 1749, 2750, 5000]
bin_names = ["Small(<1750)", "Medium (1750-2750)", "Large (>2750)"]

school_size_bin_df = school_student_calc_df
school_size_bin_df = school_size_bin_df.assign(size_bracket = pd.cut(school_size_bin_df["size"], bins, labels=bin_names))


In [94]:
# Creating a summary DataFrame using values calculated in school_size_bin_df
school_spending_df = pd.DataFrame({"School Size Bracket": school_size_bin_df["size_bracket"],
                                   "School Name":  school_size_bin_df["school_name"],
                                   "Average Math Score": list(map('{:.2f}'.format,school_size_bin_df["math_average"])),
                                   "Average Reading Score": list(map('{:.2f}'.format,school_size_bin_df["reading_average"])),
                                   "% Passing Math":  list(map('{:.2f}%'.format,school_size_bin_df["math_pass_percent"])),
                                   "% Passing Reading":  list(map('{:.2f}%'.format,school_size_bin_df["reading_pass_percent"])),
                                   "% Overall Passing Rate": list(map('{:.2f}%'.format,school_size_bin_df["overall_passing_rate"]))                                   
                                  })
school_spending_df

,School Size Bracket,School Name,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,Large (>2750),Bailey High School,77.05,81.03,66.68%,81.93%,74.31%
1,Medium (1750-2750),Cabrera High School,83.06,83.98,94.13%,97.04%,95.59%
2,Large (>2750),Figueroa High School,76.71,81.16,65.99%,80.74%,73.36%
3,Medium (1750-2750),Ford High School,77.10,80.75,68.31%,79.30%,73.80%
4,Small(<1750),Griffin High School,83.35,83.82,93.39%,97.14%,95.27%
5,Large (>2750),Hernandez High School,77.29,80.93,66.75%,80.86%,73.81%
6,Small(<1750),Holden High School,83.80,83.81,92.51%,96.25%,94.38%
7,Large (>2750),Huang High School,76.63,81.18,65.68%,81.32%,73.50%
8,Large (>2750),Johnson High School,77.07,80.97,66.06%,81.22%,73.64%
9,Small(<1750),Pena High School,83.84,84.04,94.59%,95.95%,95.27%


### Scores by School Type

* Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).

In [80]:
# Creating a summary DataFrame using values calculated in school_student_calc_df
school_type_df = pd.DataFrame({"School Name":  school_student_calc_df["school_name"],
                                   "Average Math Score": list(map('{:.2f}'.format,school_student_calc_df["math_average"])),
                                   "Average Reading Score": list(map('{:.2f}'.format,school_student_calc_df["reading_average"])),
                                   "% Passing Math":  list(map('{:.2f}%'.format,school_student_calc_df["math_pass_percent"])),
                                   "% Passing Reading":  list(map('{:.2f}%'.format,school_student_calc_df["reading_pass_percent"])),
                                   "% Overall Passing Rate": list(map('{:.2f}%'.format,school_student_calc_df["overall_passing_rate"])),
                                   "School Type": school_student_calc_df["type"]
                                  })
school_type_df



,School Name,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,School Type
0,Bailey High School,77.05,81.03,66.68%,81.93%,74.31%,District
1,Cabrera High School,83.06,83.98,94.13%,97.04%,95.59%,Charter
2,Figueroa High School,76.71,81.16,65.99%,80.74%,73.36%,District
3,Ford High School,77.10,80.75,68.31%,79.30%,73.80%,District
4,Griffin High School,83.35,83.82,93.39%,97.14%,95.27%,Charter
5,Hernandez High School,77.29,80.93,66.75%,80.86%,73.81%,District
6,Holden High School,83.80,83.81,92.51%,96.25%,94.38%,Charter
7,Huang High School,76.63,81.18,65.68%,81.32%,73.50%,District
8,Johnson High School,77.07,80.97,66.06%,81.22%,73.64%,District
9,Pena High School,83.84,84.04,94.59%,95.95%,95.27%,Charter
